In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import requests
import random

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [4]:
# 获取近期所有计划
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 45
                AND opt_status = 1
                AND create_time>=date( NOW() - INTERVAL 1440 HOUR )
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [5]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 45 
            AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df

In [6]:
def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df

In [7]:
# 获取近期优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.chl_user_id 'channel_id',
            a.source_id,
            a.image_id,
            b.creative_info 
        FROM
            (
            SELECT
                a.chl_user_id,
                a.source_id,
                a.image_id,
                b.ad_id,
                b.ad_account_id 
            FROM
                db_ptom.ptom_plan a
                LEFT JOIN db_ptom.ptom_third_plan b ON a.plan_id = b.plan_id 
            WHERE
                a.media_id = 45 
                AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
                AND a.game_id IN ({}) 
                AND b.ad_id IS NOT NULL 
            ) a
            INNER JOIN db_ptom.ptom_third_ad_creative b ON a.ad_id = b.ad_id 
            AND a.ad_account_id = b.ad_account_id 
        WHERE
            b.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.media_id = 45 
            AND a.image_id IS NOT NULL
            

    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result

In [8]:
def get_plan_json(plan_info):
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)

    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)

    temp = plan_info['ocpc'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ocpc', axis=1, inplace=True)

    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'create_time',
           'bidtype', 'producttypes', 'ftypes', 'pause', 'autoExpansion',
           'education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
           'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
           'device', 'age', 'transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
           'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
           'deepTransType', 'roiRatio', 'useRoi']]
    
    # plan_info 去重
    plan_info = plan_info.sort_values(by='create_time')
    plan_info = plan_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
    return plan_info

In [9]:
plan_info = get_plan_info()
plan_info = get_plan_json(plan_info)
creative_info = get_creative()
creative_info = creative_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
creative_info['creative_info'] = creative_info['creative_info'].apply(json.loads)
temp = creative_info['creative_info'].apply(pd.Series)
if 'image_id' in temp.columns:
    temp = temp.drop('image_id', axis=1)
temp = temp[['materialstyle','material']]
temp['material'] = temp['material'].apply(json.loads)
creative_info = pd.concat([creative_info,temp], axis=1)
creative_info.drop('creative_info', axis=1, inplace=True)
temp = creative_info['material'].apply(pd.Series)
temp = temp[['brand','elements','title','subtitle']]
creative_info = pd.concat([creative_info,temp], axis=1)
creative_info.drop('material', axis=1, inplace=True)
creative_info_1 = creative_info[creative_info['elements'].isna()]
creative_info_2 = creative_info[creative_info['elements'].notna()]
if creative_info_1.shape[0] != 0:
    creative_info_1['titles'] = creative_info_1['title'].apply(lambda x: [x])
creative_info_2['titles'] = creative_info_2['elements'].apply(lambda x: x['titles'])
creative_info = pd.concat([creative_info_1,creative_info_2],axis=0)
creative_info.drop(['elements','title'],axis=1,inplace=True)
creative_info['subtitle'] = creative_info['subtitle'].fillna(creative_info['subtitle'].mode()[0])

plan_info = pd.merge(plan_info, creative_info, on=['channel_id','source_id'], how='inner')
image_info = get_image_info()
launch_report = get_launch_report()

In [874]:
# plan_info.drop(['endtime','ad_id','ad_group_id','ad_name','media_id','plan_id','platform','launch_op_id','amount','paper_amount',
#                'click_num','pv','active_num','create_role_num','new_user_num','bid_mode','bid_status','bid','bid2','budget_mode',
#                 'budget','ad_start_time','ad_end_time','opt_status','status','syn_time','remark','update_time','origin','ad_resource_id',
#                 'image_id','inventory_type','raise_status'],axis=1, inplace=True)
# plan_info.drop(['campaignFeedId','adgroupFeedId','starttime','adgroupFeedName'],axis=1, inplace=True)
# plan_info.drop(['monitorUrl','urlType','isManualBidForMaxMode','appTransId','isManualDeepBidForMaxMode','lpUrl','crowd',
#                 'excludeCrowd','directories','mediaCategories','place','mediaCategoriesBindType','net','app','bizArea','articleType',
#                 'customArea','lifeStage'
#                ],axis=1, inplace=True)

### 构造新计划

In [875]:
# 获取近5天回款率高的素材
def get_good_image():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.image_id AS 'image_id',
            a.image_name AS 'image_name',
            a.media_id,
            a.label_ids,
            ifnull( a.image_run_date_amount, 0 ) AS 'image_run_date_amount',
            ifnull( a.image_create_role_pay_num, 0 ) AS 'image_create_role_pay_num',
            ifnull( a.image_create_role_num, 0 ) AS 'image_create_role_num',
            ifnull( a.image_create_role_pay_sum, 0 ) AS 'image_create_role_pay_sum',
            ifnull( a.image_source_num, 0 ) AS 'image_source_num',
            (
            CASE

                    WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                    0 ELSE IFNULL( a.image_create_role_pay_num, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                END 
                ) AS 'image_create_role_pay_rate',
                (
                CASE

                        WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                    END 
                    ) AS 'image_create_role_cost',
                    (
                    CASE

                            WHEN ifnull( a.image_create_role_pay_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_pay_num, 0 ) 
                        END 
                        ) AS 'image_create_role_pay_cost',
                        (
                        CASE

                                WHEN ifnull( a.image_run_date_amount, 0 )= 0 THEN
                                0 ELSE IFNULL( a.image_create_role_pay_sum, 0 ) / ifnull( a.image_run_date_amount, 0 ) 
                            END 
                            ) AS 'image_create_role_roi' 
                        FROM
                            (
                            SELECT
                                aa.image_id AS 'image_id',
                                aa.image_name AS 'image_name',
                                aa.media_id AS 'media_id',
                                aa.label_ids AS 'label_ids',
                                sum( aa.amount ) AS 'image_run_date_amount',
                                IFNULL( sum( aa.create_role_num ), 0 ) AS 'image_create_role_num',
                                IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'image_create_role_pay_num',
                                IFNULL( sum( bb.new_role_money ), 0 ) AS 'image_create_role_pay_sum',
                                IFNULL( sum( aa.image_source_num ), 0 ) AS 'image_source_num' 
                            FROM
                                (
                                SELECT
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id,
                                    a.media_id,
                                    c.image_id,
                                    c.image_name,
                                    c.label_ids,
                                    IFNULL( sum( a.amount ), 0 ) AS amount,
                                    IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                                    count( DISTINCT b.plan_id ) AS 'image_source_num' 
                                FROM
                                    db_stdata.st_lauch_report a
                                    INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                    AND a.source_id = b.source_id 
                                    AND a.channel_id = b.chl_user_id
                                    LEFT JOIN db_data_ptom.ptom_image_info c ON b.image_id = c.image_id 
                                WHERE
                                    a.tdate_type = 'day' 
                                    AND a.tdate <= date( NOW()) AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 5 DAY ) 
                                    AND a.amount > 0 
                                    AND a.media_id IN ( 45 ) 
                                    AND b.image_id IS NOT NULL 
                                    AND b.image_id <> '' 
                                    AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL ) 
                                GROUP BY
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id 
                                HAVING
                                    c.image_id IS NOT NULL 
                                    AND c.image_name <> '' 
                                ) aa
                                LEFT JOIN (
                                SELECT
                                    game_id,
                                    channel_id,
                                    source_id,
                                    IFNULL( sum( m.new_role_money ), 0 ) AS new_role_money,
                                    IFNULL( sum( m.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                FROM
                                    (
                                    SELECT
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id,
                                        IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                                        IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_lauch_report a
                                        INNER JOIN db_data_ptom.ptom_plan pp ON ( a.game_id = pp.game_id AND a.channel_id = pp.chl_user_id AND a.source_id = pp.source_id ) 
                                    WHERE
                                        a.tdate = date(
                                        NOW()) 
                                        AND a.tdate_type = 'day' 
                                    GROUP BY
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id 
                                    HAVING
                                        ( new_role_money > 0 OR pay_role_user_num > 0 ) UNION ALL
                                    SELECT
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id,
                                        sum( c.create_role_money ) new_role_money,
                                        IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_game_days c 
                                    WHERE
                                        c.report_days = 5 
                                        AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                                        AND c.tdate_type = 'day' 
                                        AND c.query_type = 13 
                                    GROUP BY
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id 
                                    HAVING
                                    ( new_role_money > 0 OR pay_role_user_num > 0 )) m 
                                GROUP BY
                                    game_id,
                                    channel_id,
                                    source_id 
                                ) bb ON aa.game_id = bb.game_id 
                                AND aa.channel_id = bb.channel_id 
                                AND aa.source_id = bb.source_id 
                            GROUP BY
                                aa.image_id,
                                aa.image_name,
                                aa.media_id 
                            HAVING
                            image_run_date_amount >= 100 
            ) a
            WHERE a.image_create_role_pay_sum >0 and 
                a.media_id = 45
                ORDER BY image_create_role_roi Desc
    '''
#     finalSql = sql.format(game_id)
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    result_df = result_df[['image_id','image_run_date_amount','image_create_role_pay_num','image_create_role_pay_sum',
                        'image_create_role_pay_cost','image_create_role_roi']]
    result_df['roi_rank'] = result_df['image_create_role_roi'].rank(ascending=False)
#     result_df = result_df[result_df['roi_rank'] <= np.ceil((result_df.shape[0] / 1.5))]
    result_df = result_df[result_df['roi_rank']<=7]
    return result_df['image_id'].values

In [10]:
# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_data_ptom.ptom_plan a 
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45 
            AND b.game_id IN ({}) 
            AND b.amount >= 200 
            AND b.platform = 1
            AND b.pay_role_user_num >= 1 
            AND b.new_role_money >= 6
            AND (b.new_role_money / b.amount)>=0.001
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [11]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id=45 and score>=510 and dt=CURRENT_DATE group by image_id,label_ids'
    
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [12]:
def get_score_image_other():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id in (16,10) and score>=590 and dt=CURRENT_DATE group by image_id,label_ids'
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [13]:
def filter_violation_image(image_ids):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND NOT FIND_IN_SET(1001706, game_ids)
    '''
    cur.execute(sql.format(image_ids))
    result = pd.read_sql(sql.format(image_ids), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [24]:
def filter_game_image(image_ids, game_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND FIND_IN_SET({}, game_ids)
    '''
    cur.execute(sql.format(image_ids, game_id))
    result = pd.read_sql(sql.format(image_ids, game_id), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [14]:
def get_now_plan_roi_other():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_data_ptom.ptom_plan a 
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id in (45,16,10,32) 
            AND b.game_id IN ({}) 
            AND b.amount >= 200 
            AND b.pay_role_user_num >= 1 
            AND b.new_role_money >= 6
            AND (b.new_role_money / b.amount)>=0.001
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [15]:
plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
plan_info_current = plan_info[plan_info['create_time']>=pd.datetime.now() - pd.DateOffset(30)]
image_info = image_info[image_info['image_id'].notna()]
image_info['image_id'] = image_info['image_id'].astype(int)

In [16]:
now_plan_roi_other = get_now_plan_roi_other()

In [17]:
now_plan_roi_other

,ad_account_id,channel_id,source_id,tdate,amount,new_role_money,roi,pay_rate
0,11614,21478,1428826,2022-04-30,1231.32,39.0,0.031673,0.1176
72,11391,21206,1428435,2022-04-30,673.67,6.0,0.008906,0.1429
71,11147,21206,1427349,2022-04-30,2339.67,6.0,0.002564,0.0196
70,11149,21206,1427348,2022-04-30,2344.90,12.0,0.005117,0.1250
69,11146,21206,1427347,2022-04-30,2413.24,42.0,0.017404,0.1000
68,11145,21206,1427345,2022-04-30,2353.65,7.0,0.002974,0.0526
67,11147,21206,1427340,2022-04-30,2362.51,6.0,0.002540,0.0152
66,11147,21206,1427335,2022-04-30,1622.20,6.0,0.003699,0.0526
65,11148,21206,1427334,2022-04-30,2816.08,104.0,0.036931,0.0156
64,11146,21206,1403397,2022-04-30,2216.07,6.0,0.002707,0.0556


In [882]:

# df_create_other = pd.merge(plan_info_current,image_info,on=['channel_id','source_id','image_id'],how='left')
# df_create_other = pd.merge(df_create_other,now_plan_roi_other.drop(['ad_account_id'], axis=1),on=['channel_id','source_id'],how='inner')

In [25]:
# now_plan_roi = pd.merge(now_plan_roi,creative_info,on=['channel_id','source_id'],how='left')
# now_plan_roi = now_plan_roi[now_plan_roi['image_id'].notna()]
# score_image_1 = get_good_image()
score_image_1 = get_score_image_other()
score_image_2 = get_score_image()
score_image = np.union1d(score_image_1,score_image_2)

In [26]:
score_image

array([38407, 40046, 40728, 41343, 42463, 42830, 42912, 42955, 43255,
       43345, 43401, 43439, 43512, 43538, 43641, 43883, 43984, 44101,
       44176, 44178, 44270, 44370, 44392, 44487, 44588, 44692, 44723,
       44725, 44785, 44816, 44824, 44853, 44961, 44969, 45061, 45087,
       45092, 45137, 45164, 45188, 45197, 45198, 45203, 45218, 45221,
       45230, 45250, 45268, 45293, 45331, 45361, 45362, 45375, 45460,
       45466, 45491, 45494, 45511, 45517, 45520], dtype=int64)

In [27]:
score_image = filter_game_image(score_image, 1001794)
# score_image = np.intersect1d(df_create_other['image_id'].unique(), score_image)

In [28]:
score_image

array([38407, 40046, 40728, 41343, 42463, 43439, 43883, 44101, 44176,
       44178, 44370, 44392, 44723, 44816, 44824, 45460, 45466, 45491,
       45494, 45517], dtype=int64)

In [886]:
now_plan_roi = get_now_plan_roi()
df_create = pd.merge(plan_info_current,image_info,on=['channel_id','source_id','image_id'],how='left')
df_create = pd.merge(df_create,now_plan_roi.drop(['ad_account_id'], axis=1),on=['channel_id','source_id'],how='inner')
# df_create['platform'] = df_create['platform'].astype(str)
# df_create['platform'] = df_create['platform'].map({"['ANDROID']":1,"['IOS']":2})
# df_create['platform'] = df_create['platform'].astype(int)
df_create.to_csv('./df_create.csv',index=0)
df_create.shape

(46, 44)

In [887]:
# df_create['optimizeDeepTrans'].value_counts()

In [888]:
# 只跑付费ROI，不跑激活付费
df_create = df_create[df_create['optimizeDeepTrans']==False]

In [889]:
df_create['game_id'].value_counts()

1001794    45
1001961     1
Name: game_id, dtype: int64

In [890]:
def create_plan(df_create,score_image):
    # 选ad_account_id、image_id每个账号+素材8条
    game_id = 1001794  ## 选择包：
#     df_create = df_create[df_create['game_id'].isin([1001794, 1001862, 1001783, 1001863, 1001961, 1001935])]
#     image_id_group = np.intersect1d(df_create['image_id'].unique(), score_image)
    image_id_group = filter_violation_image(score_image)
    
    print(image_id_group)
#     df_create = df_create[df_create['image_id'].isin(image_id_group)]
    
    # 选择预算，不限制预算
    plan = pd.DataFrame({'image_id': image_id_group})
    
    sample_df = df_create[['producttypes', 'ftypes','education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
                       'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
                       'device', 'age','transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
                   'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
                   'deepTransType', 'roiRatio', 'useRoi', 'materialstyle',
                   'brand', 'titles', 'subtitle']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    plan = plan.reset_index(drop=True)
    plan['game_id'] = game_id
    return plan

In [891]:
# image_id_group = np.intersect1d(df_create['image_id'].unique(), score_image)
# print(image_id_group)

In [892]:
plan_create = create_plan(df_create,score_image)

[40046 42463 42682 42958 43439 43790 43883 44101 44178]


In [893]:
label_map = {10507:['220', '199,220', '220,239', '117', '132,136', '119', ',220', '140', '114,156', '121,199,220','119,239'],
            10508:['220', '199,220', '148', '115,148', ',220', '201', '117', '159',
                   '132,170', '128,140,197', '220,115', '117,115', '117,128', '135,197',
                   '121,159,194,195,201', '117,220,221', '117,239', '220,239', '140',
                   '121,199,220'],
            10509:['220', '199,220', '159', '132', '148', '133,132', '201', '132,136',
                   '117', '140', ',220', '220,239', '201,159,194,195', '121,122,159,195',
                   '135,136,132', '115', '131,135,136', '117,239', '202,220,199', '148,239'],
            10510:['220', '159', '199,220', '117', '220,239', '132', ',220', '117,239',
                   '201,159,194,195', '121,122,159,195', '133,132', '132,170',
                   '129,135,170', '117,220,221', '132,133', '199,220,236'],
            10511:['220', '135,132', '199,220', ',220', '117', '201', '220,239',
                   '128,140,197', '128,132,134', '133,132,197', '117,134,130'],
            11368:['220', '199,220', '220,239', '117', '132,136', '119', ',220', '140', '114,156', '121,199,220','119,239'],
            11369:['220', '199,220', '148', '115,148', ',220', '201', '117', '159',
                   '132,170', '128,140,197', '220,115', '117,115', '117,128', '135,197',
                   '121,159,194,195,201', '117,220,221', '117,239', '220,239', '140',
                   '121,199,220'],
            11370:['220', '199,220', '159', '132', '148', '133,132', '201', '132,136',
                   '117', '140', ',220', '220,239', '201,159,194,195', '121,122,159,195',
                   '135,136,132', '115', '131,135,136', '117,239', '202,220,199', '148,239'],
            11371:['220', '159', '199,220', '117', '220,239', '132', ',220', '117,239',
                   '201,159,194,195', '121,122,159,195', '133,132', '132,170',
                   '129,135,170', '117,220,221', '132,133', '199,220,236'],
            11372:['220', '135,132', '199,220', ',220', '117', '201', '220,239',
                   '128,140,197', '128,132,134', '133,132,197', '117,134,130']}

In [894]:
def get_image_label():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
            a.image_id,
            a.label_ids
        FROM
            db_data_ptom.ptom_image_info a
        WHERE
          a.create_time >= '2021-01-01'
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df

In [895]:
plan_create = pd.merge(plan_create, get_image_label(), on=['image_id'])

In [896]:
plan_create['label_ids'].value_counts()

220        3
148        2
148,239    2
117        2
Name: label_ids, dtype: int64

In [897]:
# 检验标签是否都在列表中
label_list = []
for key,val in label_map.items():
    label_list += val
    label_list = list(set(label_list))

for label in plan_create['label_ids'].value_counts().index:
    print(label, label in label_list)
    

220 True
148 True
148,239 True
117 True


In [898]:
plan_create.shape

(9, 32)

In [899]:
ad_account_id_group = np.array([10507, 10508, 10509, 10510, 10511, 11368, 11369, 11370, 11371, 11372])
# num_pre = int(plan_create.shape[0] / len(ad_account_id_group))

plan_result = pd.DataFrame()
for ad_account_id in ad_account_id_group:
    plan_create_ad = plan_create[plan_create['label_ids'].isin(label_map[ad_account_id])]
    print(plan_create_ad.shape)
    plan_num = 6
    if plan_create_ad.shape[0] < plan_num:
        plan_num = plan_create_ad.shape[0]
    plan_result_ = plan_create_ad.sample(n=plan_num, replace=False).reset_index(drop=True)
#     plan_create = plan_create.drop(plan_create[plan_create['image_id'].isin(plan_result_.image_id.values)].index, axis=0)
    plan_result_['ad_account_id'] = ad_account_id
    plan_result = plan_result.append(plan_result_)   
plan_result['autoExpansion'] = 0
plan_result['pause'] = False
plan_result.drop('label_ids', axis=1, inplace=True)
plan_result.shape

(5, 32)
(7, 32)
(9, 32)
(5, 32)
(5, 32)
(5, 32)
(7, 32)
(9, 32)
(5, 32)
(5, 32)


(54, 34)

In [900]:
plan_result = plan_result.reset_index(drop=True)
plan_result['deepTransTypes'] = plan_result.apply(lambda x: [26],axis=1)
plan_result['brand'] = plan_result.apply(lambda x: '罗马：单机加强版' ,axis=1)
# plan_result['plan_auto_task_id'] = "12545"
plan_result['plan_auto_task_id'] = "12550,12549"
plan_result['op_id'] = 13268
plan_result['flag'] = 'BD'
plan_result['game_name'] = '帝国'
plan_result['ocpcBid'] = plan_result['ocpcBid'].apply(lambda x: random.randint(5200.00, 5400.00))
plan_result['roiRatio'] = 0.03

In [901]:
ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, ['autoExpansion','education', 'customAge', 'keywords',
       'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
       'excludeTrans', 'region', 'device', 'age']].to_json()))
plan_result['audience'] = ad_info
plan_result.drop(['autoExpansion','education', 'customAge', 'keywords',
       'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
       'excludeTrans', 'region', 'device', 'age'], axis=1, inplace=True)
ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, [ 'transType', 'ocpcLevel',
       'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
       'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi']].to_json()))
plan_result['ocpc'] = ad_info
plan_result.drop([ 'transType', 'ocpcLevel',
       'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
       'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi'], axis=1, inplace=True)
plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志

In [902]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"
    open_api_url_prefix = "https://ptom.caohua.com/"
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 45
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [903]:
get_ad_create(plan_result)

结束....


{'code': 200, 'data': True, 'msg': ''}